In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pw
import pandas as pd
#from requests import exceptions

from sklearn import preprocessing
from sklearn.decomposition import PCA
from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from bokeh.io import output_notebook
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
client_id = pw.client_id
client_secret = pw.client_secret
redirect_uri = pw.redirect_uri
user_id = pw.user_id
username = pw.username

In [3]:
token = util.prompt_for_user_token(user_id,
                                   'playlist-read-collaborative',
                                   client_id=client_id,
                                   client_secret=client_secret,
                                   redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

In [4]:
def getPlaylists(userName):
    
    playlists = sp.user_playlists(userName)
    playlist_lst = []

    for playlist in playlists['items']:
        playlist_dict = {}
        playlist_dict['playlistName'] = playlist['name']
        playlist_dict['playlistId'] = playlist['id']
        playlist_dict['playlistHref'] = playlist['href']
        playlist_dict['playlisOwner'] = playlist['owner']['display_name']
        playlist_dict['playlisPublic'] = playlist['public']
        playlist_dict['tracksHref'] = playlist['tracks']['href']
        playlist_dict['tracksCount'] = playlist['tracks']['total']

        playlist_lst.append(playlist_dict)
    
    return pd.DataFrame.from_dict(playlist_lst)

getPlaylists('kocza.milan')

,playlistName,playlistId,playlistHref,playlisOwner,playlisPublic,tracksHref,tracksCount
0,fuk.society,309rQiuXlHQ1WqwkHouoJO,https://api.spotify.com/v1/playlists/309rQiuXl...,Milán Kócza,True,https://api.spotify.com/v1/playlists/309rQiuXl...,10
1,Részegen hazafelé,4tEeo0kzKunhBwsPYndOBz,https://api.spotify.com/v1/playlists/4tEeo0kzK...,Milán Kócza,True,https://api.spotify.com/v1/playlists/4tEeo0kzK...,48
2,cool hits,79huJWUFredOrxzcXdCGYo,https://api.spotify.com/v1/playlists/79huJWUFr...,Milán Kócza,True,https://api.spotify.com/v1/playlists/79huJWUFr...,192
3,lépegetős,6eDPUoJUsnA0580PYCXdTX,https://api.spotify.com/v1/playlists/6eDPUoJUs...,Milán Kócza,True,https://api.spotify.com/v1/playlists/6eDPUoJUs...,25
4,British Mood Top 250,0xlgkjKX7eUFYxeVRVWA7t,https://api.spotify.com/v1/playlists/0xlgkjKX7...,Bence Vitárius,True,https://api.spotify.com/v1/playlists/0xlgkjKX7...,280
5,Grime Shutdown,37i9dQZF1DWSOkubnsDCSS,https://api.spotify.com/v1/playlists/37i9dQZF1...,Spotify,True,https://api.spotify.com/v1/playlists/37i9dQZF1...,79


In [5]:
def getTracks(playlistId):
    
    sp = spotipy.Spotify(auth=token)
    track_lst = []
    sourcePlaylist = sp.user_playlist(user=username, playlist_id=playlistId)
    tracks = sourcePlaylist['tracks']
    
    while True:
        sp = spotipy.Spotify(auth=token)
        for track in tracks['items']:
            
            track_dict = {}

            artistId = track['track']['album']['artists'][0]['id']
            artistInfo = sp.artist(artistId)

            songId = track['track']['id']
            audioInfo = sp.audio_features(songId)[0]

            track_dict['playListId'] = playlistId
            #track_dict['playListName'] = playlist["playlistName"]
            track_dict['added_at'] = track['added_at']
            track_dict['artisId'] = artistId
            track_dict['artisName'] = track['track']['album']['artists'][0]['name']
            track_dict['artisType'] = track['track']['album']['artists'][0]['type']
            track_dict['genre'] = artistInfo['genres']
            track_dict['artistPopularity'] = artistInfo['popularity']
            track_dict['albumId'] = track['track']['album']['id']
            track_dict['albumName'] = track['track']['album']['name']
            track_dict['albumReleaseDate'] = track['track']['album']['release_date']
            track_dict['albumImage'] = track['track']['album']['images'][0]['url']
            track_dict['songId'] = songId
            track_dict['songTitle'] = track['track']['name']
            track_dict['songPopularity'] = track['track']['popularity']
            track_dict['songDurationMs'] = track['track']['duration_ms']
            track_dict['danceability'] = audioInfo['danceability']
            track_dict['energy'] = audioInfo['energy']
            track_dict['loudness'] = audioInfo['loudness']
            track_dict['mode'] = audioInfo['mode']
            track_dict['speechiness'] = audioInfo['speechiness']
            track_dict['acousticness'] = audioInfo['acousticness']
            track_dict['instrumentalness'] = audioInfo['instrumentalness']
            track_dict['liveness'] = audioInfo['liveness']
            track_dict['valence'] = audioInfo['valence']
            track_dict['tempo'] = audioInfo['tempo'] 

            track_lst.append(track_dict)

        if tracks['next'] == None:
            break

        tracks = sp._get(tracks['next'])
    
    return pd.DataFrame.from_dict(track_lst)

df = getTracks('79huJWUFredOrxzcXdCGYo')
df

,playListId,added_at,artisId,artisName,artisType,genre,artistPopularity,albumId,albumName,albumReleaseDate,...,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,79huJWUFredOrxzcXdCGYo,2017-06-01T16:43:45Z,4kI8Ie27vjvonwaB2ePh8T,Portugal. The Man,artist,"[indie pop, indie rock, modern rock, rock]",75,2fbps64P1Z4EEktydBn9yZ,Feel It Still,2017-03-02,...,0.780,0.798,-5.151,0,0.0613,0.0406,0.000063,0.0733,0.771,78.969
1,79huJWUFredOrxzcXdCGYo,2017-06-02T10:19:22Z,5a0etAzO5V26gvlbmHzT9W,Nicolas Jaar,artist,"[downtempo, electronica, microhouse]",61,2EvZiOMBlC9b5hbjbZCjZv,Sirens,2016-09-30,...,0.583,0.409,-15.926,0,0.3920,0.8020,0.240000,0.1390,0.186,89.887
2,79huJWUFredOrxzcXdCGYo,2017-06-04T09:22:49Z,6nxWCVXbOlEVRexSbLsTer,Flume,artist,"[australian dance, australian electropop, aust...",80,0rIkPxL8Uv4zb78irBAuwS,Flume (Deluxe Edition),2013-11-11,...,0.436,0.627,-6.703,0,0.0274,0.2360,0.120000,0.3640,0.135,169.998
3,79huJWUFredOrxzcXdCGYo,2017-06-04T09:38:05Z,0cmWgDlu9CwTgxPhf403hb,Bonobo,artist,"[downtempo, electronica, jazztronica, ninja, n...",71,7sN6lCgPf1rbZYmA3edMKv,The North Borders,2013-03-21,...,0.640,0.844,-8.412,0,0.0374,0.3950,0.933000,0.0827,0.364,119.042
4,79huJWUFredOrxzcXdCGYo,2017-06-10T14:31:43Z,2AcUPzkVWo81vumdzeLLRN,Rhye,artist,"[downtempo, indie soul, indietronica, shimmer ...",69,3kSUJ193A42dfs82PmOUP0,Woman,2013-01-01,...,0.652,0.621,-12.733,1,0.0380,0.3420,0.651000,0.1770,0.750,104.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,79huJWUFredOrxzcXdCGYo,2019-12-21T11:12:54Z,4VN0CM9QYQRvgIQyZJF9Ym,228k,artist,[],33,0w4ybYTcpViTM8WuK7P23x,Chantal,2017-03-28,...,0.582,0.498,-9.549,0,0.0301,0.2160,0.006160,0.0884,0.131,108.006
188,79huJWUFredOrxzcXdCGYo,2019-12-21T12:11:04Z,5Lhxlge1CR1DrgDAje8Qaw,NoMBe,artist,"[electropop, indie poptimism, indie soul, indi...",66,5evhGOdsU34OjKur2PPUgw,They Might've Even Loved Me (Re:Imagination),2019-03-06,...,0.752,0.466,-10.928,0,0.0429,0.0300,0.060100,0.1600,0.243,104.996
189,79huJWUFredOrxzcXdCGYo,2019-12-22T09:33:10Z,3TBfn8PtXr42OrTPaoC3E8,The Futureheads,artist,"[alternative dance, british indie rock, dance-...",49,5WBxcZbQ2mxSnamfAN3mHa,This Is Not The World,2008-05-25,...,0.421,0.989,-2.612,1,0.1540,0.0017,0.000000,0.3010,0.371,155.988
190,79huJWUFredOrxzcXdCGYo,2019-12-25T22:27:23Z,56UIK6wmbbxuQ6VA20vrWD,Lola Marsh,artist,"[israeli indie, new americana]",60,6LibM0mzGnod1oIlMuxPnt,What Am I,2019-12-06,...,0.319,0.437,-11.122,0,0.0354,0.7240,0.533000,0.0927,0.461,86.377


In [8]:
features = ['acousticness', 'danceability', 'liveness', 'loudness', 'speechiness']
loudness = df[["loudness"]].values
loudness_scaled = preprocessing.minmax_scale(loudness)

df['loudness'] = pd.DataFrame(loudness_scaled)
x = df.loc[:, features].values

In [11]:
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k, random_state=0)
    km = km.fit(x)
    sum_of_squared_distances.append(km.inertia_)
    
#sum_of_squared_distances

output_notebook()

p = figure(plot_width=500, plot_height=400)

p.line(K, sum_of_squared_distances)
p.xaxis.axis_label = 'nr of clusters'
p.yaxis.axis_label = 'sum of squared distances'

show(p)

Loading BokehJS ...

In [18]:
for n_clusters in range(2,15):
    clusterer = KMeans (n_clusters=n_clusters)
    preds = clusterer.fit_predict(x)
    centers = clusterer.cluster_centers_

    score = silhouette_score (x, preds, metric='euclidean')
    print ("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))

For n_clusters = 2, silhouette score is 0.39004664125627614)
For n_clusters = 3, silhouette score is 0.24701450882959022)
For n_clusters = 4, silhouette score is 0.26920802016650774)
For n_clusters = 5, silhouette score is 0.2469874848780197)
For n_clusters = 6, silhouette score is 0.21963154490110934)
For n_clusters = 7, silhouette score is 0.22073481357631344)
For n_clusters = 8, silhouette score is 0.22003579455040315)
For n_clusters = 9, silhouette score is 0.23112688243925103)
For n_clusters = 10, silhouette score is 0.2245411045535457)
For n_clusters = 11, silhouette score is 0.20866444987264732)
For n_clusters = 12, silhouette score is 0.2113892868940375)
For n_clusters = 13, silhouette score is 0.22298994526369306)
For n_clusters = 14, silhouette score is 0.21531257348152757)


In [12]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(x)
y_kmeans = kmeans.predict(x)

In [13]:
#PCA
pca = PCA(n_components=2)
pcaComponents = pca.fit_transform(x)

print("Explained variance ratio: " + str(sum(pca.explained_variance_ratio_)*100) + "%" )

Explained variance ratio: 69.4480665036894%


In [15]:
pcadf = pd.DataFrame(data = pcaComponents, columns=['pca1', 'pca2'])
pcadf['label'] = y_kmeans

finaldf = pd.concat([pcadf, df[['albumName', 'albumImage', 'artisName', 'songTitle']]], axis=1)
finaldf["label"] = finaldf["label"].astype(str)
finaldf

,pca1,pca2,label,albumName,albumImage,artisName,songTitle
0,-0.242772,-0.206852,2,Feel It Still,https://i.scdn.co/image/ab67616d0000b273e2fdc4...,Portugal. The Man,Feel It Still
1,0.691837,-0.029480,0,Sirens,https://i.scdn.co/image/ab67616d0000b27349475e...,Nicolas Jaar,No
2,-0.083640,0.233474,1,Flume (Deluxe Edition),https://i.scdn.co/image/ab67616d0000b273f470c8...,Flume,Bring You Down
3,0.145723,-0.077122,2,The North Borders,https://i.scdn.co/image/ab67616d0000b273b03308...,Bonobo,Cirrus
4,0.188493,-0.088589,2,Woman,https://i.scdn.co/image/ab67616d0000b273b4a8b9...,Rhye,The Fall
...,...,...,...,...,...,...,...
187,0.007555,-0.099546,2,Chantal,https://i.scdn.co/image/ab67616d0000b273eceadd...,228k,Chantal
188,-0.128772,-0.210115,2,They Might've Even Loved Me (Re:Imagination),https://i.scdn.co/image/ab67616d0000b273c3d8bd...,NoMBe,Wait 2.0
189,-0.380267,0.188306,1,This Is Not The World,https://i.scdn.co/image/ab67616d0000b273a3ccd5...,The Futureheads,The Beginning Of The Twist
190,0.490763,0.156999,0,What Am I,https://i.scdn.co/image/20459f9728a1c98e2c49dc...,Lola Marsh,What Am I


In [23]:
finaldf.to_csv("test.csv")

In [19]:
output_notebook()

Loading BokehJS ...

In [30]:
output_notebook()

source = ColumnDataSource(data=dict(
    x = finaldf["pca1"],
    y = finaldf["pca2"],
    album = finaldf["albumName"],
    artist = finaldf["artisName"],
    title = finaldf["songTitle"],
    imgs = finaldf["albumImage"],
    category = finaldf["label"]
))

tooltips = """
    <div>        
        <div>
        <p style="font-size: 12px; font-weight: bold;">
        <img src="@imgs" alt="@imgs" style="float:left; width:42px; height:42px; border=1 margin: 0px 15px 15px 0px;">
        @artist-@title </p>
        </div>
        <div>
        <p>@category<p/>
        <div/>
        
    </div>
"""


index_cmap = factor_cmap('category', palette = ['red', 'yellow', 'blue'], factors=sorted(finaldf["label"].unique()))

p = figure(plot_width=800, plot_height=600, tooltips=tooltips)

p.circle("x", "y", size=25, alpha=0.5, source=source,  fill_color=index_cmap)



show(p)

Loading BokehJS ...

array(['2', '0', '1'], dtype=object)